# Konlpy 설치

- jdk SE Development 설치
- JPype 다운(python 버전, 시스템 64/32 확인)
- wd로 JPype 파일 이동


In [ ]:
# !pip install --upgrade pip
# !pip install JPype1-1.0.2-cp38-cp38-2in_amd64.whl
# !pip install konlpy
#  --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org



In [1]:
import os
os.getcwd()

'C:\\Users\\admin\\Desktop\\완친파 전체 연습문제 소스코드 (1)'

# Crwaling

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd

In [3]:
df = pd.DataFrame(columns= ['번호','제목','등록일','조회수','링크', '내용'])
df

,번호,제목,등록일,조회수,링크,내용


In [5]:
driver = webdriver.Chrome("C:/temp/chromedriver_240/chromedriver.exe")
driver.get('https://www.kcomwel.or.kr/kcomwel/cust/free/free.jsp')

In [6]:
# 필요시 시작 페이지로 이동
# driver.find_element_by_xpath('//a[.//span[text()=%s]]'%6).click()

In [9]:
for n in range(1,37):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    voice = soup.select('body > #wrap > #scontainer-wrap > #content > #sub-cont > #is-cont >div.cont-box > div > div> div > div > div.jwxe_root.jwxe_board.free-board.ko > div.tb-scroll.jwxe_root.jwxe_board.common-board.ko > div.mscroll > table > tbody > tr')
    
    for i in range(len(voice)):
        no = voice[i].find_all('td')[0].text.strip() # 번호
        title = voice[i].find_all('td')[1].text.strip() # 제목
        time = voice[i].find_all('td')[2].text.strip() # 등록일
        view = voice[i].find_all('td')[3].text.strip() # 조회수
        a = voice[i].find('a')['href'] # 링크
        
        # 게시글 클릭
        driver.find_element_by_css_selector('#is-cont > div.cont-box > div > div > div > div > div.jwxe_root.jwxe_board.free-board.ko > div.mscroll > table > tbody > tr:nth-child(%s) > td.tdleft > a'%(i+1)).click()
        content = BeautifulSoup(diver.page_source, 'html.parser').select('#article_text')[0].get_text(' ', strip =True)
        
        content = content.replace('\xa0', '')
        driver.implicitly_wait(3)
        
        # 목록 클릭
        driver.find_element_by_css_selector('#is-cont > div.cont-box > div > div > div > div > div.jwxe_root.jwxe_board.free-board.ko > div.btn_wrap.al_right > a:nth-child(1)').click()
        
        new_data = [no, title, time, view, a , content]
        df.loc[len(df)] = new_data
        print(10*(n-1)+i+1, '번째 게시글 : ',new_data, '\n\n')
        
        
    print('-----------------------------', n, '페이지 완료 ---------------------------------------------\n')
    
    driver.implicitly_wait(2)
    
    driver.find_element_by_xpath('//a[.//span[text()=%s]]'% str(n+1) ).click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#is-cont > div.cont-box > div > div > div > div > div.jwxe_root.jwxe_board.free-board.ko > div.mscroll > table > tbody > tr:nth-child(1) > td.tdleft > a"}
  (Session info: chrome=86.0.4240.75)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape 

In [11]:
df1 = pd.read_csv('./data/근로복지공단 자유게시판(1-56).csv' , encoding=('euc-kr'))
df2 = pd.read_csv('./data/근로복지공단 자유게시판(58-60).csv' , encoding=('euc-kr'))
df3 = pd.read_csv('./data/근로복지공단 자유게시판(61-350).csv' , encoding=('euc-kr'))
df4 = pd.read_csv('./data/근로복지공단 자유게시판(351-358).csv' , encoding=('euc-kr'))

print(df1.shape, df2.shape, df3.shape, df4.shape)

FileNotFoundError: [Errno 2] No such file or directory: './data/근로복지공단 자유게시판(1-56).csv'

In [ ]:
df = pd.concat([df1,df2,df3,df4], axis= 0)

df.shape

In [ ]:
df.info()

In [ ]:
df.등록일 = pd.to_datetime(df.등록일)

df.info()

In [ ]:
# df.to_csv('./data/근로복지공단 자유게시판(통합).csv' , encoding = 'euc-kr')

# EDA & Wordcloud

In [ ]:
import konlpy
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

plt.rc('font', family= 'NanumGothic')
%matplotlib.inline

In [ ]:
df = pd.read_csv('./data/근로복지공단 자유게시판(통합).csv' , encoding = 'euc-kr')
df.shape

In [ ]:
df.describe()

In [ ]:
print('최근 게시물 : ', df.등록일.max())
print("첫 게시물 : ", df.등록일.min())



# 제목

In [ ]:
text = ' '.join(df.제목)

okt = Okt()
noun = okt.nouns(text)
len(noun)

In [ ]:
# 한글자 제거
noun = [i for i in noun if len(i)>=2]
len(noun)

In [ ]:
count = Counter(noun)
noun_list= count.most_common(100)
noun_list[:10]

In [ ]:
noun_list_rm = []

for i in range(len(noun_list)):
    if noun_list[i][0] not in['근로', '복지', '공단']:
        noun_list_rm.append(noun_list[i])
        
noun_list_rm[:10]


In [ ]:
wordcloud = WordCloud(font_path="",
                     background_color ='white',
                     max_words =100,
                     colormap='nipy_spectral').generate_from_frequencies(dict(noun_list_rm))


fig = plt.figure(figsize=(50,50))

plt.axis('off')
plt.imshow(wordcloud)



# Word Network

In [ ]:
import re

text = ' '.join(df.내용)
list_kor = re.compile('[가-힣]+').findall(text)

list_kor[:10]


In [ ]:
len(list_kor)

In [ ]:
text_kor = ' '.join(list_kor)
okt = Okt()

noun = okt.nouns(text_kor)
len(noun)

In [ ]:
# 한글자 제거
noun = [i for i in noun if len(i)>=2]
len(noun)

In [ ]:
count = Counter(noun)
noun_list = count.most_common(100)
noun_list[:10]




In [ ]:
# 관련 키워드 삭제
noun_list_rm =[]

for i in range(len(noun_list)):
    if noun_list[i][0] not in['근로', '복지', '공단']:
        noun_lsit_rm.append(noun_list[i])
        
noun_list_rm[:10]

In [ ]:
wordcloud = WordCloud(font_path="",
                     background_color ='white',
                     max_words =100,
                     colormap='nipy_spectral').generate_from_frequencies(dict(noun_list_rm))


fig = plt.figure(figsize=(50,50))

plt.axis('off')
plt.imshow(wordcloud)

